In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""
import sys

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.models.ranking import *
from twinews.evaluation.utils import *

In [3]:
from nlptools.basics import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [4]:
from twinews.yfnotebooks.convnet.graph import Graph
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from twinews.yfnotebooks.load_data import load_char_data,char_index  # load data tools

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
isNotebook = '__file__' not in locals()
TEST = isNotebook

In [6]:
logger = Logger(tmpDir('logs') + "/Convnet.log") if isNotebook else Logger("Convnet-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [7]:
modelName = "Convnet"

# config

In [9]:
config = \
{
    'splitVersion': 2,
    'testVersion':2, # with some parameters changed
    
    # for input data
    'maxUsers': 10 if TEST else None, # Sub-sampling
    'maxDocuments': None,
    'useExtraNews': 0 if TEST else None, # None = unlimited, 0 = no extra news
    'minDF': 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 2000
    'maxDF': 20, # Remove top 20 voc elements
    'lowercase': True,
    'doLemmatization': True,
    
   # for model    
    'seq_length' : 100,
#     'char_embedding_size': 100,
    'learning_rate': 0.0005,
    'keep_prob': 0.7,
    'vocab_size': 3029,
    'class_size': 2,
    'epochs': 100,
    'batch_size': 1000,
    
    # for ouput
    'similarity': 'cosine',

#     'historyRef': 0.3, # 1, 1.0, 0.5, 0.3, 3, 10
}

# get data

In [10]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0,
                       maxUsers=config['maxUsers'])
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 43.19s | message: Eval data loaded


{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


--> toc total duration: 43.42s | message: Got Twinews evaluation data


In [11]:
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)

In [12]:
newsList = trainNewsList + testNewsList
log(str(len(newsList)) + " urls for newsList:\n" + b(newsList), logger=logger)

9690 urls for newsList:
[
  https://buff.ly/2yxBJKY,
  https://www.bloomberg.com/news/articles/2017-12-13/teva-s-reported-job-cuts-prompt-strike-threat-fro,
  ...,
  http://www.baltimoresun.com/news/opinion/readersrespond/bs-ed-rr-conowingo-letter-20180111-story.htm,
  https://www.nytimes.com/2018/01/12/opinion/proud-nation-of-holers.html
]


In [13]:
# get all the titles of news urls link to form (url: title) dict
titles = getNewsTitles(newsList)

twinews news (version 1.0) initialised.
  100% [====================] (total duration: 5m 24.79s, mean duration: 0.033s)


In [14]:
# tokenize the tiltles since there was no tokenized title in the database
titlesToken = []
for i in pb(list(range(len(titles))), logger=logger, message="tokenizing"):
    titlesToken.append(wordTokenize(titles[i]))
bp(titlesToken,2)

tokenizing   0% [                    ]
tokenizing  10% [==                  ] (5.579s left)
tokenizing  20% [====                ] (3.08s left)
tokenizing  30% [======              ] (2.1s left)
tokenizing  40% [========            ] (1.544s left)
tokenizing  50% [==========          ] (1.16s left)
tokenizing  60% [============        ] (0.86s left)
tokenizing  70% [==============      ] (0.604s left)
tokenizing  80% [================    ] (0.385s left)
tokenizing  90% [==================  ] (0.185s left)
tokenizing 100% [====================] (total duration: 1.8s, mean duration: 0s)


[ [ 24, Excellent, ..., Eater, SF ], [ Teva, Braces, ..., -, Bloomberg ], ..., [ Exelon, must, ..., Baltimore, Sun ], [ Opinion, |, ..., York, Times ] ]


In [15]:
for i in pb(list(range(len(titlesToken))), logger=logger, message="Lower casing"):
    if titlesToken[i] == None:
        #print(titlesToken[i])  # just to see how many urls have empty title
        continue
    else:
        for u in range(len(titlesToken[i])):
            titlesToken[i][u] = titlesToken[i][u].lower()
bp(titlesToken, logger)

Lower casing   0% [                    ]
Lower casing  10% [==                  ] (0.09s left)
Lower casing  20% [====                ] (0.12s left)
Lower casing  30% [======              ] (0.116s left)
Lower casing  40% [========            ] (0.105s left)
Lower casing  50% [==========          ] (0.09s left)
Lower casing  60% [============        ] (0.066s left)
Lower casing  70% [==============      ] (0.051s left)
Lower casing  80% [================    ] (0.035s left)
Lower casing  90% [==================  ] (0.017s left)
Lower casing 100% [====================] (total duration: 0.17s, mean duration: 0s)
[ [ 24, excellent, ..., eater, sf ], [ teva, braces, ..., -, bloomberg ], ..., [ exelon, must, ..., baltimore, sun ], [ opinion, |, ..., york, times ] ]


In [16]:
# lemmatization
lemmatizer = WordNetLemmatizer()
pbar = ProgressBar(len(titlesToken), logger=logger, message="Lemmatization")
for i in range(len(titlesToken)):
    if titlesToken[i] == None:
        # print(titlesToken[i])  # same as last one
        continue
    else:
        for u in range(len(titlesToken[i])):
            titlesToken[i][u] = lemmatizer.lemmatize(titlesToken[i][u])
    pbar.tic()
bp(titlesToken, logger)

Lemmatization   0% [                    ]
Lemmatization  10% [==                  ] (9.27s left)
Lemmatization  20% [====                ] (4.36s left)
Lemmatization  30% [======              ] (2.66s left)
Lemmatization  40% [========            ] (1.799s left)
Lemmatization  50% [==========          ] (1.25s left)
Lemmatization  60% [============        ] (0.866s left)
Lemmatization  70% [==============      ] (0.582s left)
Lemmatization  80% [================    ] (0.352s left)
Lemmatization  90% [==================  ] (0.162s left)
[ [ 24, excellent, ..., eater, sf ], [ teva, brace, ..., -, bloomberg ], ..., [ exelon, must, ..., baltimore, sun ], [ opinion, |, ..., york, time ] ]


In [17]:
# build a dict with url as key, and tokenized title as value
urlTitles= dict()
for i in range(len(titlesToken)):
    urlTitles[newsList[i]] = titlesToken[i]
bp(urlTitles, 2, logger)

{
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: [ america, 's, ..., wall, st. ],
  http://247wallst.com/special-report/2018/01/11/25-best-fitness-tech-gadgets/: [ 25, best, ..., wall, st. ],
  http://a.msn.com/00/en-us/AAuyJLc?ocid=st: [ what, living, ..., motley, fool ],
  http://a.msn.com/00/en-us/BBHoy0Q?ocid=st: [ the, world, ..., in, 2017 ],
  http://a.msn.com/01/en-ie/BBHzIAS?ocid=st: [ rare, supermoon, ..., see, it ],
  ...,
  https://www.zerohedge.com/news/2018-01-08/credit-card-debt-hits-all-time-high-consumers-unleash-historic-shopping-spree: [ credit, card, ..., zero, hedge ],
  https://www.zerohedge.com/news/2018-01-08/grand-jury-empaneled-10-million-fraud-probe-involving-jane-and-bernie-sanders: [ grand, jury, ..., zero, hedge ],
  https://www.zerohedge.com/news/2018-01-08/its-not-his-job-illinois-treasurer-plays-activist-college-savers-cash: [ ", it, ..., zero, hedge ],
  https://yaledailynews.com/blog/2018/01/06/yale-psychiatrist-met-wi

# load the vocab

In [18]:
# You need to change the file path, I put it under ../Twinews/twinews/yfnotebooks/title_vocab.txt
# to filter out the words not in the vacab to shrink down the size of the query
titleVocab = []
with open('/home/yuting/PycharmProjects/data/title_vocab.txt','r') as file:
    for line in file.readlines():
        titleVocab.append(line.strip('\n'))

bp(titleVocab,2)

[ !, #, ..., •, … ]


In [19]:
len(trainUsers.keys())

10

# build user history

In [20]:
# for multiple users
trainUserQuery = dict()
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="build user history")

# for each user
for usr in trainUsers.keys():
    queryFinal = []
    
    # choose 15 urls in their history to form the query text
    if len(list(trainUsers[usr])) < 15:
        query = list(trainUsers[usr])
    else:
        query = random.sample(list(trainUsers[usr]),15)
        
    # put 15 titles together, using extend
    temp = []
    for url in query:
        if urlTitles[url] == None:
            continue
        else:
            temp.extend(urlTitles[url])
            
    # the following two steps are making the query titles more simplified and efficient        
    # filter what is not in the vocab
    tempCompact = []
    for i in range(len(temp)):
        if temp[i] in titleVocab:
            tempCompact.append(temp[i])
    
    # remove duplicated & detokenize
    queryCompact = detokenize(list(set(tempCompact)))
    
    queryFinal.append(queryCompact)
    trainUserQuery[usr] = queryFinal
    
    pbar.tic()

build user history  10% [==                  ] (0.36s left)
build user history  20% [====                ] (0.319s left)
build user history  30% [======              ] (0.303s left)
build user history  40% [========            ] (0.255s left)
build user history  50% [==========          ] (0.21s left)
build user history  60% [============        ] (0.166s left)
build user history  70% [==============      ] (0.128s left)
build user history  80% [================    ] (0.084s left)
build user history  90% [==================  ] (0.042s left)
build user history 100% [====================] (total duration: 0.42s, mean duration: 0.041s)


# rankings

In [21]:
# I'm not sure whether you can use this checkpoint file
def computeSimilarity(p,h,y):
    
    tf.reset_default_graph() 
    model = Graph()
    saver = tf.train.Saver()
    
    # restore the trianed model
    with tf.Session()as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt')
        loss, acc, prediction = sess.run([model.loss, model.acc, model.prediction],
                             feed_dict={model.p: p,
                                        model.h: h,
                                        model.y: y,
                                        model.keep_prob: 1})

        #print('loss: ', loss, ' acc:', acc)
        return prediction
#logit = computeSimilarity(p,h,y)

In [127]:
# rankings of the candidates with logits(second last uotput) results
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="testing")
rankings = dict()
for usr in trainUsers.keys():
    usrRankings = []
    
    # get historical data as query(p)
    query = trainUserQuery[usr] * 1000
    
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        # get 1000 h as a list(h)
        h_temp = []
        for url in candidates:
            h_temp.append(detokenize(urlTitles[url]))
        # get p,h
        p,h = char_index(query,h_temp)
        y = np.zeros((1000))                    # whatever initialization
        
        scoresCandidates = []
        logit = computeSimilarity(p,h,y)            # get the 1000 scores list
        scoresCandidates = zip(candidates,list(logit[:,1]))
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings
    
    pbar.tic()

In [22]:
# rankings of the candidates with 0/1 classification results
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="testing")
rankings = dict()
for usr in trainUsers.keys():
    usrRankings = []
    
    # get historical data as query(p)
    query = trainUserQuery[usr] * 1000
    
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        # get 1000 h as a list(h)
        h_temp = []
        for url in candidates:
            h_temp.append(detokenize(urlTitles[url]))
        # get p,h
        p,h = char_index(query,h_temp)
        y = np.zeros((1000))                    # whatever initialization
        
        scoresCandidates = []
        prediction = computeSimilarity(p,h,y)            # get the 1000 scores list
        scoresCandidates = zip(candidates,prediction)
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings
    
    pbar.tic()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt


testing  10% [==                  ] (5m 16.079s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt


testing  20% [====                ] (4m 47.48s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt


...

testing  80% [================    ] (1m 16.802s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt


testing  90% [==================  ] (38.579s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt


testing 100% [====================] (total duration: 6m 27.829s, mean duration: 38.783s)


# add the ranking

In [23]:
# check the ranking before adding to the db for evaluation
checkRankings(rankings,evalData['candidates'],maxUsers=None)

In [24]:
# add the ranking into de DB
addRanking(modelName, rankings, config, logger=logger)